# Numerical Methods

## Lecture 6: Numerical Linear Algebra II

### Exercise solutions

### <span style="color:blue">Exercise 6.1: lower-triangular matrices</span>

Convince yourselves of the following facts:


* The multiplication of arbitrary lower-triangular square matrices is also lower-triangular.


* $L_2(L_1(L_0A)) = U \implies A = L_0^{-1}(L_1^{-1}(L_2^{-1}U))$


* and hence that $A=LU$ where $U$ is the upper-triangular matrix found at the end of Guassian elimination, and where $L$ is the 
following  matrix
$$ L = L_0^{-1}L_1^{-1}L_2^{-1} $$


* Finally, compute this product of these lower-triangular matrices to show that 
$$L = 
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{black}{1}} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{black}{4}} & {\color{black}7} & {\color{black}1} & {\color{black}0}\\
    {\color{black}{5}} & {\color{black}8} & {\color{black}2} & {\color{black}1}\\   
  \end{bmatrix}
$$
i.e. that the multiplication of these individual atomic matrices (importantly in this order) simply merges the entries from the non-zero columns of each atomic matrix, and hence is both lower-triangular, as well as trivial to compute.

In [1]:
import numpy as np
from scipy import linalg
from pprint import pprint

# as above: A matrix,
A=np.array([[5., 7. , 5., 9.],
            [5., 14.,   7.,  10.],
            [20., 77., 41., 48.],
            [25., 91.,  55., 67.]])
# lower triangular matrices,
L0 = np.array([[1,0,0,0],[-1,1,0,0],[-4,0,1,0],[-5,0,0,1]])
L1 = np.array([[1,0,0,0],[0,1,0,0],[0,-7,1,0],[0,-8,0,1]])
L2 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,-2,1]])
# and their inverse matrices
L0_ = linalg.inv(L0)
L1_ = linalg.inv(L1)
L2_ = linalg.inv(L2)

# our base matrix
pprint(A)

# ad: The multiplication of arbitrary lower-triangular square matrices is also lower-triangular
pprint(np.dot(L0, L1))
pprint(np.dot(L2, L1))

# ad: L2(L1(L0 A)) = U
U = np.dot(L2, np.dot(L1, np.dot(L0, A)))
pprint(U)

# ad: A = L0^-1(L1^-1(L2^-1 U))
A = np.dot(L0_, np.dot(L1_, np.dot(L2_, U))) 
pprint(A)

# ad: L = L0^-1 L1^-1 L2^-1
L = np.dot(L0_,np.dot(L1_,L2_))
pprint(L)

# ad: A = LU
# numpy.allclose takes two numpy arrays and checks whether all elements are identical to some tolerance
np.allclose(A, np.dot(L, U))

array([[ 5.,  7.,  5.,  9.],
       [ 5., 14.,  7., 10.],
       [20., 77., 41., 48.],
       [25., 91., 55., 67.]])
array([[ 1,  0,  0,  0],
       [-1,  1,  0,  0],
       [-4, -7,  1,  0],
       [-5, -8,  0,  1]])
array([[ 1,  0,  0,  0],
       [ 0,  1,  0,  0],
       [ 0, -7,  1,  0],
       [ 0,  6, -2,  1]])
array([[5., 7., 5., 9.],
       [0., 7., 2., 1.],
       [0., 0., 7., 5.],
       [0., 0., 0., 4.]])
array([[ 5.,  7.,  5.,  9.],
       [ 5., 14.,  7., 10.],
       [20., 77., 41., 48.],
       [25., 91., 55., 67.]])
array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [4., 7., 1., 0.],
       [5., 8., 2., 1.]])


True

### <span style="color:blue">Exercise 6.2: LU decomposition</span>

Starting from your Gaussian elimination code produce a new code to compute the LU decomposition of a matrix. First, store $L$ and $U$ in two different matrices; you could then try a version where you store the entries of both $L$ and $U$ in $A$ as described above.

In [2]:
import numpy as np
import scipy.linalg as sl
from pprint import pprint

def LU_dec(A):
    # upper triangular matrix contains gaussian elimination result
    # we won't change A in-place but create a local copy
    A = A.copy()
    m, n = A.shape
    # lower triangular matrix has identity diagonal and scaling factors
    # start from the identity:
    L = np.identity(n)
    # Loop over each pivot row.
    for k in range(n):
        # Loop over each equation below the pivot row.
        for i in range(k+1, n):
            # Define the scaling factor outside the innermost
            # loop otherwise its value gets changed.
            s = (A[i,k]/A[k,k])
            for j in range(k, n):
                A[i,j] = A[i,j] - s*A[k,j]
            # scaling factors make up the lower matrix 
            L[i,k] = s
    # A now is the upper triangular matrix U
    return L, A


A = np.array([[ 5., 7.,   5.,  9.],
              [ 5., 14.,  7., 10.],
              [20., 77., 41., 48.],
              [25., 91. ,55., 67.]])

L, U = LU_dec(A)

pprint(A)
pprint(L)
pprint(U)
np.allclose(np.dot(L,U), A)

array([[ 5.,  7.,  5.,  9.],
       [ 5., 14.,  7., 10.],
       [20., 77., 41., 48.],
       [25., 91., 55., 67.]])
array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [4., 7., 1., 0.],
       [5., 8., 2., 1.]])
array([[5., 7., 5., 9.],
       [0., 7., 2., 1.],
       [0., 0., 7., 5.],
       [0., 0., 0., 4.]])


True

### <span style="color:blue">Exercise 6.3: Partial pivoting</span>

Implement partial pivoting.

In [3]:
import numpy as np
from pprint import pprint

# Part of Ex. 3.3. This function is not necessary but makes the LU_dec_pp function less cluttered
# Making general operations into functions is good way of improving readability and reducing your workload later!
def swap_rows(A,j,k):
    B = np.copy(A[j,:])
    C = np.copy(A[k,:])
    # second label, BUT NOT CREATING A 2ND ARRAY. Try it for yourself if you want.
    A[k,:] = B
    A[j,:] = C
    return A

# Ex. 3.3 A function to perform LU decomposition with partial pivoting
def LU_dec_pp(A):
    m, n = A.shape
    A = A.copy() # we won't modify in place but create local copy
    P_ = np.identity(m)
    L  = np.identity(m)
    for k in range(m):
        j = np.argmax(abs(A[k:,k])) 
        # Find the index of the largest ABSOLUTE value. np.argmax will return 
        # the index of the largest element in an array
        j+= k               # A[1+2,2] = A[3,2] = 3!        
        A  = swap_rows(A,j,k)
        P_ = swap_rows(P_,j,k)
        for j in range(k+1,m):
            s = A[j,k]/A[k,k]
            A[j,k:] -= A[k,k:]*s  
            L[j,k] = s
    U = A   
    return P_.T, L, U

A = np.array([[ 5., 7.,   5.,  9.],
              [ 5., 14.,  7., 10.],
              [20., 77., 41., 48.],
              [25., 91. ,55., 67.]])

P, L, U = LU_dec_pp(A)
pprint(A)
pprint(L)
pprint(U)
pprint(np.dot(P, np.dot(L,U)))
np.allclose(np.dot(P, np.dot(L,U)), A)

array([[ 5.,  7.,  5.,  9.],
       [ 5., 14.,  7., 10.],
       [20., 77., 41., 48.],
       [25., 91., 55., 67.]])
array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.2       ,  1.        ,  0.        ,  0.        ],
       [ 0.8       , -0.375     ,  1.        ,  0.        ],
       [ 0.2       ,  0.375     ,  0.33333333,  1.        ]])
array([[ 25.        ,  91.        ,  55.        ,  67.        ],
       [  0.        , -11.2       ,  -6.        ,  -4.4       ],
       [  0.        ,   0.        ,  -5.25      ,  -7.25      ],
       [  0.        ,   0.        ,   0.        ,   0.66666667]])
array([[ 5.,  7.,  5.,  9.],
       [ 5., 14.,  7., 10.],
       [20., 77., 41., 48.],
       [25., 91., 55., 67.]])


True

Note the solution was given at the end of the solutions to last week's lecture.